In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import types as T
import os

In [2]:
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [3]:
def find_csv_files(folder_path):
    csv_files = []
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith('.csv'):
                csv_files.append(os.path.join(root, file))
    return csv_files

In [4]:
def show(df, limit=10):
    return df.toPandas().head(limit)

In [5]:
folder = r"C:\Development\ultimateNakMuay\data\processed"
files = find_csv_files(folder)

In [6]:
files = files[2:]

In [7]:
organizations = ["ufc", "onefc", "bellator", "glory"]

organizations = {
    k:{
        "event": spark.read.csv(list(filter(lambda x: "event" in x and k in x, files))[0], header=True),
        "result": spark.read.csv(list(filter(lambda x: "result" in x and k in x, files))[0], header=True),
        }
    for k in organizations
}

In [8]:
def get_data(org):
    return organizations[org]["event"], organizations[org]["result"]

In [97]:
event, result = get_data("onefc")

In [107]:
event = event.withColumn("ejc1", F.element_at(F.split(F.col("event"), ":"), 1))\
    .withColumn("event", F.regexp_replace(F.col("event"), "ONE:", "ONE Championship:"))\
    .withColumn("ejc2", F.trim(F.element_at(F.split(F.col("event"), ":"), -1)))

In [108]:
result = result.withColumn("event_name", F.regexp_replace(F.col("event_name"), "Road to ONE:", "Road to ONE Championship:"))\
    .withColumn("rjc1", F.trim(F.element_at(F.split(F.col("event_name"), ":"), -1)))

In [109]:
join_conditions = (
    (event.event == result.event_name)
    |(event.ejc1 == result.event_name)
    |(event.ejc2 == result.rjc1)
    )
dbg = event.select("event", "ejc1", "ejc2").distinct().join(
    result.select("event_name", "rjc1").distinct(), join_conditions, "full_outer"
)

In [121]:
show(dbg.filter(F.col("event_name").isNull() & F.col("event").contains("Warrior")), 1000)

,event,ejc1,ejc2,event_name,rjc1
0,ONE Warrior Series 10,ONE Warrior Series 10,ONE Warrior Series 10,None,None
1,Road to ONE 8: Night of Warriors,Road to ONE 8,Night of Warriors,None,None
2,ONE Warrior Series 4,ONE Warrior Series 4,ONE Warrior Series 4,None,None
3,ONE Warrior Series 5,ONE Warrior Series 5,ONE Warrior Series 5,None,None
4,ONE Warrior Series 1,ONE Warrior Series 1,ONE Warrior Series 1,None,None
5,ONE Championship: Winter Warriors II,ONE Championship,Winter Warriors II,None,None
6,ONE Warrior Series 11,ONE Warrior Series 11,ONE Warrior Series 11,None,None
7,ONE Warrior Series 6,ONE Warrior Series 6,ONE Warrior Series 6,None,None
8,ONE Warrior Series 8,ONE Warrior Series 8,ONE Warrior Series 8,None,None
9,ONE Warrior Series 2,ONE Warrior Series 2,ONE Warrior Series 2,None,None


In [123]:
show(dbg.filter(F.col("event").isNull()))

,event,ejc1,ejc2,event_name,rjc1
0,None,None,None,ONE Championship: Masters Of Fate,Masters Of Fate
1,None,None,None,ONE Fight Night 14,ONE Fight Night 14
2,None,None,None,ONE Championship: Age Of Dragons,Age Of Dragons
3,None,None,None,2011 in ONE Championship,2011 in ONE Championship
4,None,None,None,ONE Championship: Beyond The Horizon,Beyond The Horizon
5,None,None,None,Road to ONE Championship: Night of Warriors 17,Night of Warriors 17
6,None,None,None,ONE Championship: Dynasty of Heroes,Dynasty of Heroes
7,None,None,None,ONE Championship: Winter Warriors 2,Winter Warriors 2
8,None,None,None,ONE Championship: Edge Of Greatness,Edge Of Greatness
9,None,None,None,Road to ONE Championship: RUF 40,RUF 40
